# Introduction

We use this paper as support in order to prepare data sets: 


Yu, H., Sun, X., Thordarson, D.S., Solbakken, K., Solvang, W.D. (2024). A Network-Based Set Covering Model for Charging Station Location Problem: A Case Study in Norway. In: Wang, Y., Yu, T., Wang, K. (eds) Advanced Manufacturing and Automation XIII. IWAMA 2023. Lecture Notes in Electrical Engineering, vol 1154. Springer, Singapore. https://doi.org/10.1007/978-981-97-0665-5_26

In [23]:
import urllib.request
import requests
import json
import numpy as np

In [24]:
api_key = "API_KEY"

# 1. Set up data and convert to JSON

https://learn.microsoft.com/en-us/bingmaps/rest-services/routes/calculate-a-distance-matrix?redirectedfrom=MSDN

(link to API documentation)

In [25]:
data = {
    "origins": [
    {"latitude": 68.57, "longitude": 14.91},
    {"latitude": 68.70, "longitude": 15.42},
    {"latitude": 68.62, "longitude": 15.65},
    {"latitude": 68.53, "longitude": 15.73},
    {"latitude": 68.42, "longitude": 15.99},
    {"latitude": 68.56, "longitude": 16.24},
    {"latitude": 68.63, "longitude": 16.59},
    {"latitude": 68.50, "longitude": 16.70},
    {"latitude": 68.53, "longitude": 16.99},
    {"latitude": 68.54, "longitude": 17.26},
    {"latitude": 68.55, "longitude": 17.56},
    {"latitude": 68.41, "longitude": 17.42},
    {"latitude": 68.91, "longitude": 15.10},
    {"latitude": 68.73, "longitude": 15.52},
    {"latitude": 68.09, "longitude": 13.23},
    {"latitude": 68.24, "longitude": 14.56},
    {"latitude": 69.25, "longitude": 17.95}
    ],
    "destinations": [
    {"latitude": 68.57, "longitude": 14.91},
    {"latitude": 68.70, "longitude": 15.42},
    {"latitude": 68.62, "longitude": 15.65},
    {"latitude": 68.53, "longitude": 15.73},
    {"latitude": 68.42, "longitude": 15.99},
    {"latitude": 68.56, "longitude": 16.24},
    {"latitude": 68.63, "longitude": 16.59},
    {"latitude": 68.50, "longitude": 16.70},
    {"latitude": 68.53, "longitude": 16.99},
    {"latitude": 68.54, "longitude": 17.26},
    {"latitude": 68.55, "longitude": 17.56},
    {"latitude": 68.41, "longitude": 17.42},
    {"latitude": 68.91, "longitude": 15.10},
    {"latitude": 68.73, "longitude": 15.52},
    {"latitude": 68.09, "longitude": 13.23},
    {"latitude": 68.24, "longitude": 14.56},
    {"latitude": 69.25, "longitude": 17.95}
    ],
    "travelMode": "driving"
}

dataset = json.dumps(data)

# 2. Request

See the link earlier for the structure

In [26]:
url = f"https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix?key={api_key}"

response = requests.post(url, data=dataset) # HTTP request

# 3. Print result

In [ ]:
if response.status_code == 200:
    result = response.json() # need to parse to JSON response
    results = result['resourceSets'][0]['resources'][0]['results']
    for i in range (len(results)):
      indexDest = results[i]['destinationIndex']
      indexOrig = results[i]['originIndex']
      traveDist = results[i]['travelDistance']
      print(f"Distance between {indexOrig+1} and {indexDest+1} is {traveDist:.2f}.")
else:
    print("Error:", response.status_code)

# 4. Distance matrix

In [ ]:
if response.status_code == 200:
    result = response.json()
    results = result['resourceSets'][0]['resources'][0]['results']
    num_cities = 17
    distance_matrix = np.zeros((num_cities, num_cities))

    for i in range (num_cities):
      for j in range(num_cities):
          if i != j:
              traveDist = results[i * num_cities + j]['travelDistance']
              distance_matrix[i, j] = round(traveDist, 2)
else:
    print("Error:", response.status_code)

print(distance_matrix)

# 5. Binary radius matrix
_Reminder : the binary matrix $a_{ij}$_ has a 1 on position $i,j$ if city $d(i,j)\leq r$ and has a 0 otherwise  

In [29]:
def create_radius_matrix(D, r):
    RM = np.zeros_like(D)
    for i in range(D.shape[0]):
        for j in range(D.shape[1]):
            if D[i, j] <= r:
                RM[i, j] = 1
    return RM

R = 130  # example radius used in Hao Yu's paper
radius_matrix = create_radius_matrix(distance_matrix, R)
print(radius_matrix)

[[1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0.]
 [0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0.]
 [0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0.]
 [0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0.]
 [1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
